In [1]:
from __future__ import print_function
import os
import sys

# Edit this to be the full path to the extracted Spark directory on your computer
spark_home = 'C:\Users\Machine\Desktop\Pattern\spark-1.6.1-bin-hadoop2.6'

os.environ['SPARK_HOME'] = spark_home
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python\lib\py4j-0.9-src.zip'))

In [2]:
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)

In [237]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import re
import operator

In [12]:
business = sqlContext.read.json("yelp_business.json")

In [10]:
reviews = sqlContext.read.json("yelp_review.json")

In [17]:
users = sqlContext.read.json("yelp_user.json")

In [7]:
reviews.count()

2225213

In [13]:
businessPA = business.where(business["state"]=="PA")

In [8]:
#reviews.select(reviews.business_id, 
#reviews.user_id, reviews.review_id, reviews.text, reviews.stars, reviews.date).show(n=3)

In [9]:
reviewPA = reviews.join(businessPA, reviews.business_id == businessPA.business_id, 'inner').select(reviews.business_id, 
reviews.user_id, reviews.review_id, reviews.text, reviews.stars, reviews.date)

In [ ]:
reviewPA.toPandas()

In [12]:
reviewPA.toPandas().to_csv("PAReviews.csv", encoding='utf-8')

In [9]:
businessPA = business.join(businessPA, business.business_id == businessPA.business_id, 'inner').select(business.business_id,\
                business.review_count, business.stars, business.categories, business.city, business.state, \
                business.full_address, business.latitude, business.longitude,business.name, business.attributes.Alcohol, \
                business.attributes.Ambience.casual, business.attributes.Ambience.classy, business.attributes.Ambience.divey, \
                business.attributes.Ambience.hipster, business.attributes.Ambience.intimate, business.attributes.Ambience.romantic, \
                business.attributes.Ambience.touristy, business.attributes.Ambience.trendy, business.attributes.Ambience.upscale, \
                business.attributes.Delivery, business.neighborhoods, business.open)

In [12]:
businessPA.show(4)

+--------------------+------------+-----+--------------------+----------+-----+--------------------+----------------+----------------+--------------------+-------------------+----------------------------+----------------------------+---------------------------+-----------------------------+------------------------------+------------------------------+------------------------------+----------------------------+-----------------------------+--------------------+-------------+----+
|         business_id|review_count|stars|          categories|      city|state|        full_address|        latitude|       longitude|                name|attributes[Alcohol]|attributes[Ambience][casual]|attributes[Ambience][classy]|attributes[Ambience][divey]|attributes[Ambience][hipster]|attributes[Ambience][intimate]|attributes[Ambience][romantic]|attributes[Ambience][touristy]|attributes[Ambience][trendy]|attributes[Ambience][upscale]|attributes[Delivery]|neighborhoods|open|
+--------------------+----------

In [15]:
businessPA.toPandas().to_csv("PABusiness.csv", encoding='utf-8')

In [16]:
users.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliments: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- cute: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- hot: long (nullable = true)
 |    |-- list: long (nullable = true)
 |    |-- more: long (nullable = true)
 |    |-- note: long (nullable = true)
 |    |-- photos: long (nullable = true)
 |    |-- plain: long (nullable = true)
 |    |-- profile: long (nullable = true)
 |    |-- writer: long (nullable = true)
 |-- elite: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- fans: long (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)

In [17]:
users.select(users['yelping_since']).show(10)

+-------------+
|yelping_since|
+-------------+
|      2004-10|
|      2004-10|
|      2004-10|
|      2004-10|
|      2004-10|
|      2004-10|
|      2004-10|
|      2004-10|
|      2004-10|
|      2004-10|
+-------------+
only showing top 10 rows



In [10]:
usersPA = users.select(users.user_id, users.name, users.review_count, users.average_stars, users.fans, users.votes, \
                       users.yelping_since)

In [20]:
usersPA.toPandas().to_csv("UsersPA.csv", encoding='utf-8')

In [4]:
reviews_panda = pd.DataFrame.from_csv('PAReviews.csv')

In [5]:
users_panda = pd.DataFrame.from_csv('UsersPA.csv')

In [6]:
business_panda = pd.DataFrame.from_csv('PABusiness.csv')

In [16]:
users_panda

,user_id,name,review_count,average_stars,fans,votes,yelping_since
0,18kPq7GPye-YQ3LyKyAZPw,Russel,108,4.14,70,"(245, 167, 280)",2004-10
1,rpOyqD_893cqmDAtJLbdog,Jeremy,1274,3.66,1179,"(11093, 7681, 14199)",2004-10
2,4U9kSBLuBDU391x6bxU-YA,Michael,442,3.60,100,"(732, 908, 1483)",2004-10
3,fHtTaujcyKvXglE33Z5yIw,Ken,11,4.64,2,"(5, 1, 11)",2004-10
4,SIBCL7HBkrP4llolm4SC2A,Katherine,66,3.80,4,"(13, 11, 34)",2004-10


In [8]:
business_panda.head()

,business_id,review_count,stars,categories,city,state,full_address,latitude,longitude,name,...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,0zO219KZJf-5QsRNRMmELQ,3,2.0,[],Pittsburgh,PA,3116 Banksville Rd\r\nBanksville\r\nPittsburgh...,40.400387,-80.044015,Pennzoil 10 Minute Oil Change Center,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Banksville],True
1,13aQBVZEbO5MzGrmDJ0IVw,20,4.0,"[Food, Bubble Tea, Coffee & Tea]",Pittsburgh,PA,"3800 Forbes Ave\r\nOakland\r\nPittsburgh, PA 1...",40.441542,-79.956821,Fuku Tea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True
2,3zgswf_NfBJpeAoWeTkgMA,52,4.0,"[Pubs, Bars, American (Traditional), Nightlife...",Pittsburgh,PA,"5135 Liberty Ave\r\nBloomfield\r\nPittsburgh, ...",40.458267,-79.942274,Silky's Pub,...,False,False,False,False,False,False,False,False,[Bloomfield],True
3,4S5CdZawGJfP3vtR4eAQ1g,61,3.5,"[Hotels & Travel, Event Planning & Services, H...",Pittsburgh,PA,"One Bigelow Square\r\nDowntown\r\nPittsburgh, ...",40.440430,-79.994880,Doubletree By Hilton Hotel & Suites - Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Downtown],True
4,9lRJhLrLOgh0ZGhqsjBmLA,3,5.0,"[Museums, Arts & Entertainment]",Chateau,PA,"1 Allegheny Ave\r\nNorth Side\r\nChateau, PA 1...",40.445689,-80.018228,Miniature Railroad & Village,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True


In [9]:
reviews_panda.head()

,business_id,user_id,review_id,text,stars,date
0,0zO219KZJf-5QsRNRMmELQ,kAFccaTcFR9va9uigTHCNg,#NAME?,Stopped in for an oil change and they told me ...,1,2/27/2015
1,13aQBVZEbO5MzGrmDJ0IVw,JXyPdO0yzBpYvY5N0M-Qkw,BZkbBolvBkkFZxkGcmc3rw,It was cool that you could choose ingredients ...,5,10/26/2015
2,13aQBVZEbO5MzGrmDJ0IVw,x5FusRCjF-UWlCEYz_eyJA,lDwzuSg3KWlnByOmv87BbQ,This little trickle of new restaurants into Oa...,5,10/30/2015
3,13aQBVZEbO5MzGrmDJ0IVw,KOV0BJa2z0DOs2yuT1829g,jg0JJ7ShJKXWQH3YfmLRXw,"This little shop just had their soft opening, ...",4,10/31/2015
4,13aQBVZEbO5MzGrmDJ0IVw,WCje_p2V91WDLqXcFw5bGQ,mBFPK-LeAgJ9tcfkk_bJUg,Went there twice for bubble tea. Got 1 dollar ...,3,11/2/2015


In [14]:
reviews_pa = reviews.join(businessPA, reviews.business_id == businessPA.business_id, 'inner').select(reviews.business_id,businessPA.state, 
businessPA.review_count, reviews.user_id, reviews.review_id, reviews.text, reviews.stars, reviews.date)

In [18]:
reviewsPA_extended = users.join(reviews_pa, users.user_id == reviews_pa.user_id, 'inner').select(reviews_pa.business_id,reviews_pa.state,
reviews_pa.review_count, reviews_pa.user_id, reviews_pa.review_id, reviews_pa.text, reviews_pa.stars, 
users.review_count, users.fans, users.votes.useful, users.yelping_since, reviews_pa.date)

In [19]:
reviewsPA_ordered = reviewsPA_extended.orderBy(reviewsPA_extended.business_id).toPandas()

In [21]:
reviewsPA_ordered['business_id'] = reviewsPA_ordered['business_id'].map(lambda x: x.lstrip('=+-'))

In [22]:
reviewsPA_ordered['user_id']=reviewsPA_ordered['user_id'].map(lambda x: x.lstrip('=+-'))

In [23]:
reviewsPA_ordered['review_id']=reviewsPA_ordered['review_id'].map(lambda x: x.lstrip('=+-'))

In [284]:
reviewsPA_ordered.columns.values

array(['business_id', 'state', 'review_count', 'user_id', 'review_id',
       'text', 'stars', 'review_count', 'fans', 'votes[useful]',
       'yelping_since', 'date'], dtype=object)

In [83]:
reviewsPA_ordered.to_csv('PAreviews_extended.csv', encoding='utf-8')

In [96]:
business_panda['business_id'][0]

'0zO219KZJf-5QsRNRMmELQ'

In [85]:
reviewsPA_extended = pd.DataFrame.from_csv('PAreviews_extended.csv')

In [97]:
reviewsPA_extended['review_id'][1]

'GE9olaftv8AUq477WgTdQg'

In [98]:
reviewsPA_weighted = pd.DataFrame.from_csv("PAreviews_weighted.csv")

In [99]:
reviewsPA_weighted.head()

,business_id,state,review_count,user_id,review_id,stars,review_count.1,fans,votes[useful],yelping_since,...,weighted_users,weighted_rating,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010
0,UE_y6auTgq3FXlvUMkbw,PA,133,w9cHHLIP4n2MjsuC_sRMFg,I6NZHPaFJBcoFYv2vpPiQ,3,29,0,13,2014-01,...,0.905000,2.715000,160.901667,531.276667,3.301872,0,0,0,1,1
2,UE_y6auTgq3FXlvUMkbw,PA,133,hru2IDwSyN61sgWPtJw_hA,w5yuYDMZK398oR2_yVxog,2,250,13,678,2012-06,...,1.986667,3.973333,160.901667,531.276667,3.301872,0,0,0,0,1
3,UE_y6auTgq3FXlvUMkbw,PA,133,ukrBUZXE5K1J57FI7ve_RQ,ih559vXqcEEWQxPq-HMgVA,3,131,4,220,2010-12,...,1.133333,3.400000,160.901667,531.276667,3.301872,0,0,0,0,1
4,UE_y6auTgq3FXlvUMkbw,PA,133,nYWT--flsvUsPshFR5uWw,082lGCayN_bwLyKwTdRHxw,3,48,3,94,2010-08,...,0.680000,2.040000,160.901667,531.276667,3.301872,0,0,0,0,1
5,UE_y6auTgq3FXlvUMkbw,PA,133,1YPFrlSsnab1jTtRdQEWjw,lOp472FvNeZgSi068j4og,3,26,1,73,2011-06,...,0.745000,2.235000,160.901667,531.276667,3.301872,0,0,0,0,1


In [100]:
reviewsPA_weighted = reviewsPA_weighted.reset_index()

In [104]:
reviewsPA_weighted.head()

,business_id,state,review_count,user_id,review_id,stars,review_count.1,fans,votes[useful],yelping_since,...,weighted_users,weighted_rating,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010
0,UE_y6auTgq3FXlvUMkbw,PA,133,w9cHHLIP4n2MjsuC_sRMFg,I6NZHPaFJBcoFYv2vpPiQ,3,29,0,13,2014-01,...,0.905000,2.715000,160.901667,531.276667,3.301872,0,0,0,1,1
1,UE_y6auTgq3FXlvUMkbw,PA,133,hru2IDwSyN61sgWPtJw_hA,w5yuYDMZK398oR2_yVxog,2,250,13,678,2012-06,...,1.986667,3.973333,160.901667,531.276667,3.301872,0,0,0,0,1
2,UE_y6auTgq3FXlvUMkbw,PA,133,ukrBUZXE5K1J57FI7ve_RQ,ih559vXqcEEWQxPq-HMgVA,3,131,4,220,2010-12,...,1.133333,3.400000,160.901667,531.276667,3.301872,0,0,0,0,1
3,UE_y6auTgq3FXlvUMkbw,PA,133,nYWT--flsvUsPshFR5uWw,082lGCayN_bwLyKwTdRHxw,3,48,3,94,2010-08,...,0.680000,2.040000,160.901667,531.276667,3.301872,0,0,0,0,1
4,UE_y6auTgq3FXlvUMkbw,PA,133,1YPFrlSsnab1jTtRdQEWjw,lOp472FvNeZgSi068j4og,3,26,1,73,2011-06,...,0.745000,2.235000,160.901667,531.276667,3.301872,0,0,0,0,1


In [285]:
reviewsPA_weighted.columns.values

array(['business_id', 'state', 'review_count', 'user_id', 'review_id',
       'stars', 'review_count.1', 'fans', 'votes[useful]', 'yelping_since',
       'date', 'year_weighted', 'review_count_weighted', 'fans_weighted',
       'votes_weighted', 'weighted_users', 'weighted_rating', 'users_sum',
       'ratings_sum', 'ratings_avg', 'after_Jun2015', 'after_Jan2015',
       'after_Jun2014', 'after_Jan2014', 'after_Jan2010'], dtype=object)

In [105]:
trainingPA = reviewsPA_weighted

In [ ]:
trainingPA = trainingPA[trainingPA.after_Jun2015 == 0]

In [107]:
trainingPA.head()

,business_id,state,review_count,user_id,review_id,stars,review_count.1,fans,votes[useful],yelping_since,...,weighted_users,weighted_rating,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010
0,UE_y6auTgq3FXlvUMkbw,PA,133,w9cHHLIP4n2MjsuC_sRMFg,I6NZHPaFJBcoFYv2vpPiQ,3,29,0,13,2014-01,...,0.905000,2.715000,160.901667,531.276667,3.301872,0,0,0,1,1
1,UE_y6auTgq3FXlvUMkbw,PA,133,hru2IDwSyN61sgWPtJw_hA,w5yuYDMZK398oR2_yVxog,2,250,13,678,2012-06,...,1.986667,3.973333,160.901667,531.276667,3.301872,0,0,0,0,1
2,UE_y6auTgq3FXlvUMkbw,PA,133,ukrBUZXE5K1J57FI7ve_RQ,ih559vXqcEEWQxPq-HMgVA,3,131,4,220,2010-12,...,1.133333,3.400000,160.901667,531.276667,3.301872,0,0,0,0,1
3,UE_y6auTgq3FXlvUMkbw,PA,133,nYWT--flsvUsPshFR5uWw,082lGCayN_bwLyKwTdRHxw,3,48,3,94,2010-08,...,0.680000,2.040000,160.901667,531.276667,3.301872,0,0,0,0,1
4,UE_y6auTgq3FXlvUMkbw,PA,133,1YPFrlSsnab1jTtRdQEWjw,lOp472FvNeZgSi068j4og,3,26,1,73,2011-06,...,0.745000,2.235000,160.901667,531.276667,3.301872,0,0,0,0,1


In [108]:
trainingPA.to_csv("PAreview_train.csv")

In [109]:
business_merge = dict()

for i in range(len(trainingPA['business_id'])):
    if trainingPA['business_id'][i] not in business_merge:
        business_merge[trainingPA['business_id'][i]] = 1

In [110]:
business_merge_panda = pd.DataFrame(business_merge.items(), columns=['business_id','present'])

In [111]:
business_merge_panda.head()

,business_id,present
0,shrMOQmX_SX-KualwYoAzA,1
1,c3GRR8q1h4fdNV7APLzSeQ,1
2,KxidRGk6RpyvVXibiy_DIg,1
3,OT3yVpDrDVITDCIYox8d9A,1
4,Verns0y0NTYj96SyvKgnhA,1


In [112]:
right = business_panda
left = business_merge_panda

In [113]:
business_panda_updated = left.merge(right, left_on='business_id', right_on='business_id', how='inner')

In [121]:
business_panda_updated.columns.values

array(['business_id', 'present', 'review_count', 'stars', 'categories',
       'city', 'state', 'full_address', 'latitude', 'longitude', 'name',
       'attributes[Alcohol]', 'attributes[Ambience][casual]',
       'attributes[Ambience][classy]', 'attributes[Ambience][divey]',
       'attributes[Ambience][hipster]', 'attributes[Ambience][intimate]',
       'attributes[Ambience][romantic]', 'attributes[Ambience][touristy]',
       'attributes[Ambience][trendy]', 'attributes[Ambience][upscale]',
       'attributes[Delivery]', 'neighborhoods', 'open', 'users_sum',
       'ratings_avg'], dtype=object)

In [120]:
business_panda_updated.head()

,business_id,present,review_count,stars,categories,city,state,full_address,latitude,longitude,...,attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open,users_sum,ratings_avg
0,shrMOQmX_SX-KualwYoAzA,1,7,3.0,"[Bakeries, Food]",Pittsburgh,PA,535 Smithfield St\r\nOliver Bldg\r\nDowntown\r...,40.441352,-79.998017,...,NaN,NaN,NaN,NaN,NaN,False,[Downtown],True,160.901667,3.301872
1,c3GRR8q1h4fdNV7APLzSeQ,1,40,4.5,"[Auto Repair, Automotive, Tires, Oil Change St...",Pittsburgh,PA,"4157 Murray Ave\r\nGreenfield\r\nPittsburgh, P...",40.423409,-79.928138,...,NaN,NaN,NaN,NaN,NaN,NaN,[Greenfield],True,160.901667,3.301872
2,KxidRGk6RpyvVXibiy_DIg,1,7,3.0,"[Shopping, Home Services, Building Supplies, H...",Carnegie,PA,"2100 Washington Pike\r\nCarnegie, PA 15106",40.387855,-80.094976,...,NaN,NaN,NaN,NaN,NaN,NaN,[],True,160.901667,3.301872
3,OT3yVpDrDVITDCIYox8d9A,1,5,3.5,"[Restaurants, Italian]",Pittsburgh,PA,"20 Stanwix St\r\nDowntown\r\nPittsburgh, PA 15222",40.438660,-80.005135,...,False,False,False,False,False,True,[Downtown],True,160.901667,3.301872
4,Verns0y0NTYj96SyvKgnhA,1,6,1.5,"[Optometrists, Ophthalmologists, Health & Medi...",Pittsburgh,PA,"3616 Forbes Ave\r\nOakland\r\nPittsburgh, PA 1...",40.440987,-79.957777,...,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True,160.901667,3.301872


In [122]:
del business_panda_updated['present']

In [123]:
business_panda_updated.head()

,business_id,review_count,stars,categories,city,state,full_address,latitude,longitude,name,...,attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open,users_sum,ratings_avg
0,shrMOQmX_SX-KualwYoAzA,7,3.0,"[Bakeries, Food]",Pittsburgh,PA,535 Smithfield St\r\nOliver Bldg\r\nDowntown\r...,40.441352,-79.998017,Au Bon Pain Co,...,NaN,NaN,NaN,NaN,NaN,False,[Downtown],True,160.901667,3.301872
1,c3GRR8q1h4fdNV7APLzSeQ,40,4.5,"[Auto Repair, Automotive, Tires, Oil Change St...",Pittsburgh,PA,"4157 Murray Ave\r\nGreenfield\r\nPittsburgh, P...",40.423409,-79.928138,Homer's Service Center,...,NaN,NaN,NaN,NaN,NaN,NaN,[Greenfield],True,160.901667,3.301872
2,KxidRGk6RpyvVXibiy_DIg,7,3.0,"[Shopping, Home Services, Building Supplies, H...",Carnegie,PA,"2100 Washington Pike\r\nCarnegie, PA 15106",40.387855,-80.094976,Lowe's,...,NaN,NaN,NaN,NaN,NaN,NaN,[],True,160.901667,3.301872
3,OT3yVpDrDVITDCIYox8d9A,5,3.5,"[Restaurants, Italian]",Pittsburgh,PA,"20 Stanwix St\r\nDowntown\r\nPittsburgh, PA 15222",40.438660,-80.005135,Buon Giorno Cafe,...,False,False,False,False,False,True,[Downtown],True,160.901667,3.301872
4,Verns0y0NTYj96SyvKgnhA,6,1.5,"[Optometrists, Ophthalmologists, Health & Medi...",Pittsburgh,PA,"3616 Forbes Ave\r\nOakland\r\nPittsburgh, PA 1...",40.440987,-79.957777,UPMC Eye Center,...,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True,160.901667,3.301872


In [288]:
business_panda_updated.columns.values

array(['business_id', 'review_count', 'stars', 'categories', 'city',
       'state', 'full_address', 'latitude', 'longitude', 'name',
       'attributes[Alcohol]', 'attributes[Ambience][casual]',
       'attributes[Ambience][classy]', 'attributes[Ambience][divey]',
       'attributes[Ambience][hipster]', 'attributes[Ambience][intimate]',
       'attributes[Ambience][romantic]', 'attributes[Ambience][touristy]',
       'attributes[Ambience][trendy]', 'attributes[Ambience][upscale]',
       'attributes[Delivery]', 'neighborhoods', 'open', 'users_sum',
       'ratings_avg'], dtype=object)

In [124]:
trainingPA.head()

,business_id,state,review_count,user_id,review_id,stars,review_count.1,fans,votes[useful],yelping_since,...,weighted_users,weighted_rating,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010
0,UE_y6auTgq3FXlvUMkbw,PA,133,w9cHHLIP4n2MjsuC_sRMFg,I6NZHPaFJBcoFYv2vpPiQ,3,29,0,13,2014-01,...,0.905000,2.715000,160.901667,531.276667,3.301872,0,0,0,1,1
1,UE_y6auTgq3FXlvUMkbw,PA,133,hru2IDwSyN61sgWPtJw_hA,w5yuYDMZK398oR2_yVxog,2,250,13,678,2012-06,...,1.986667,3.973333,160.901667,531.276667,3.301872,0,0,0,0,1
2,UE_y6auTgq3FXlvUMkbw,PA,133,ukrBUZXE5K1J57FI7ve_RQ,ih559vXqcEEWQxPq-HMgVA,3,131,4,220,2010-12,...,1.133333,3.400000,160.901667,531.276667,3.301872,0,0,0,0,1
3,UE_y6auTgq3FXlvUMkbw,PA,133,nYWT--flsvUsPshFR5uWw,082lGCayN_bwLyKwTdRHxw,3,48,3,94,2010-08,...,0.680000,2.040000,160.901667,531.276667,3.301872,0,0,0,0,1
4,UE_y6auTgq3FXlvUMkbw,PA,133,1YPFrlSsnab1jTtRdQEWjw,lOp472FvNeZgSi068j4og,3,26,1,73,2011-06,...,0.745000,2.235000,160.901667,531.276667,3.301872,0,0,0,0,1


In [287]:
trainingPA.columns.values

array(['business_id', 'state', 'review_count', 'user_id', 'review_id',
       'stars', 'review_count.1', 'fans', 'votes[useful]', 'yelping_since',
       'date', 'year_weighted', 'review_count_weighted', 'fans_weighted',
       'votes_weighted', 'weighted_users', 'weighted_rating', 'users_sum',
       'ratings_sum', 'ratings_avg', 'after_Jun2015', 'after_Jan2015',
       'after_Jun2014', 'after_Jan2014', 'after_Jan2010'], dtype=object)

In [125]:
after_Jan2015_grouped=trainingPA.groupby(['business_id'])['after_Jan2015'].sum().reset_index(name='after_Jan2015_count')

In [133]:
after_Jan2015_grouped.head()

,business_id,after_Jan2015_count
0,01a4YOjiBbPUBxvgX6F7VQ,0
1,04ogq23JnzauhezrWEXG_g,3
2,06SQoq5EV4XAY-yKrEtCQg,0
3,079g3KpS_7j_Zw53JmD9Vg,5
4,08Z2A6u4kmaAZYeecwA4qw,5


In [134]:
right=trainingPA
left=after_Jan2015_grouped

In [143]:
count_1 = left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [145]:
count_2 = left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [149]:
count_1.head()

,business_id,state,review_count,user_id,review_id,stars,review_count.1,fans,votes[useful],yelping_since,...,weighted_rating,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010,after_Jan2015_count
0,01a4YOjiBbPUBxvgX6F7VQ,PA,3,RJChsLqZrY3DRnaFbTXVXw,BLpxEcJWBK5kjjQHIGP-NA,5,25,0,12,2014-01,...,4.525,3.810,19.050000,5.000000,0,0,1,1,1,0
1,01a4YOjiBbPUBxvgX6F7VQ,PA,3,GolOuPl1kOw_nCck2OyrgA,OVOgzUzjuqHLdwUNofsz9w,5,7,0,10,2013-10,...,4.525,3.810,19.050000,5.000000,0,0,1,1,1,0
2,04ogq23JnzauhezrWEXG_g,PA,48,unnvLojxXbPhbqsq39GRIw,9udpA-IU2BQY-JTtM3cAxw,5,22,0,72,2008-11,...,3.400,68.275,268.116667,3.927011,0,0,0,0,1,3
3,04ogq23JnzauhezrWEXG_g,PA,48,1YPFrlSsnab1jTtRdQEWjw,h6ylqst_kpul0Gl7Pvu2Mw,4,26,1,73,2011-06,...,2.980,68.275,268.116667,3.927011,0,0,0,0,1,3
4,04ogq23JnzauhezrWEXG_g,PA,48,ZG9QD6MJhV-M61dFDOUuBw,KulLATnwy9uSgasez-NJQw,5,1,0,0,2010-06,...,4.100,68.275,268.116667,3.927011,0,0,0,0,1,3


In [146]:
del count_1['after_Jan2015_count']

In [148]:
count_1['after_Jan2015_count'] = count_2['after_Jan2015_count']

In [137]:
count_1.columns.values

array(['business_id', 'after_Jan2015_count', 'state', 'review_count',
       'user_id', 'review_id', 'stars', 'review_count.1', 'fans',
       'votes[useful]', 'yelping_since', 'date', 'year_weighted',
       'review_count_weighted', 'fans_weighted', 'votes_weighted',
       'weighted_users', 'weighted_rating', 'users_sum', 'ratings_sum',
       'ratings_avg', 'after_Jun2015', 'after_Jan2015', 'after_Jun2014',
       'after_Jan2014', 'after_Jan2010'], dtype=object)

In [150]:
after_Jun2014_grouped=trainingPA.groupby(['business_id'])['after_Jun2014'].sum().reset_index(name='after_Jun2014_count')

In [151]:
after_Jun2014_grouped.head()

,business_id,after_Jun2014_count
0,01a4YOjiBbPUBxvgX6F7VQ,2
1,04ogq23JnzauhezrWEXG_g,9
2,06SQoq5EV4XAY-yKrEtCQg,0
3,079g3KpS_7j_Zw53JmD9Vg,14
4,08Z2A6u4kmaAZYeecwA4qw,6


In [153]:
right=count_1
left=after_Jun2014_grouped

In [154]:
count_2=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [155]:
count_3=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [157]:
del count_2['after_Jun2014_count']

In [158]:
count_2['after_Jun2014_count'] = count_3['after_Jun2014_count']

In [159]:
count_2.head()

,business_id,state,review_count,user_id,review_id,stars,review_count.1,fans,votes[useful],yelping_since,...,users_sum,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010,after_Jan2015_count,after_Jun2014_count
0,01a4YOjiBbPUBxvgX6F7VQ,PA,3,RJChsLqZrY3DRnaFbTXVXw,BLpxEcJWBK5kjjQHIGP-NA,5,25,0,12,2014-01,...,3.810,19.050000,5.000000,0,0,1,1,1,0,2
1,01a4YOjiBbPUBxvgX6F7VQ,PA,3,GolOuPl1kOw_nCck2OyrgA,OVOgzUzjuqHLdwUNofsz9w,5,7,0,10,2013-10,...,3.810,19.050000,5.000000,0,0,1,1,1,0,2
2,04ogq23JnzauhezrWEXG_g,PA,48,unnvLojxXbPhbqsq39GRIw,9udpA-IU2BQY-JTtM3cAxw,5,22,0,72,2008-11,...,68.275,268.116667,3.927011,0,0,0,0,1,3,9
3,04ogq23JnzauhezrWEXG_g,PA,48,1YPFrlSsnab1jTtRdQEWjw,h6ylqst_kpul0Gl7Pvu2Mw,4,26,1,73,2011-06,...,68.275,268.116667,3.927011,0,0,0,0,1,3,9
4,04ogq23JnzauhezrWEXG_g,PA,48,ZG9QD6MJhV-M61dFDOUuBw,KulLATnwy9uSgasez-NJQw,5,1,0,0,2010-06,...,68.275,268.116667,3.927011,0,0,0,0,1,3,9


In [160]:
after_Jan2014_grouped=traindata.groupby(['business_id'])['after_Jan2014'].sum().reset_index(name='after_Jan2014_count')

In [161]:
right=count_2
left=after_Jan2014_grouped

In [162]:
count_3=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [163]:
count_4=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [166]:
del count_3['after_Jan2014_count']

In [167]:
count_3['after_Jan2014_count'] = count_4['after_Jan2014_count']

In [168]:
count_3.head()

,business_id,state,review_count,user_id,review_id,stars,review_count.1,fans,votes[useful],yelping_since,...,ratings_sum,ratings_avg,after_Jun2015,after_Jan2015,after_Jun2014,after_Jan2014,after_Jan2010,after_Jan2015_count,after_Jun2014_count,after_Jan2014_count
0,01a4YOjiBbPUBxvgX6F7VQ,PA,3,RJChsLqZrY3DRnaFbTXVXw,BLpxEcJWBK5kjjQHIGP-NA,5,25,0,12,2014-01,...,19.050000,5.000000,0,0,1,1,1,0,2,2
1,01a4YOjiBbPUBxvgX6F7VQ,PA,3,GolOuPl1kOw_nCck2OyrgA,OVOgzUzjuqHLdwUNofsz9w,5,7,0,10,2013-10,...,19.050000,5.000000,0,0,1,1,1,0,2,2
2,04ogq23JnzauhezrWEXG_g,PA,48,unnvLojxXbPhbqsq39GRIw,9udpA-IU2BQY-JTtM3cAxw,5,22,0,72,2008-11,...,268.116667,3.927011,0,0,0,0,1,3,9,12
3,04ogq23JnzauhezrWEXG_g,PA,48,1YPFrlSsnab1jTtRdQEWjw,h6ylqst_kpul0Gl7Pvu2Mw,4,26,1,73,2011-06,...,268.116667,3.927011,0,0,0,0,1,3,9,12
4,04ogq23JnzauhezrWEXG_g,PA,48,ZG9QD6MJhV-M61dFDOUuBw,KulLATnwy9uSgasez-NJQw,5,1,0,0,2010-06,...,268.116667,3.927011,0,0,0,0,1,3,9,12


In [169]:
after_Jan2010_grouped=trainingdata.groupby(['business_id'])['after_Jan2010'].sum().reset_index(name='after_Jan2010_count')
right=count_3
left=after_Jan2010_grouped

In [170]:
count_1=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [171]:
count_2=left.merge(right, left_on='business_id', right_on='business_id', how='outer')

In [173]:
del count_1['after_Jan2010_count']

In [174]:
count_1['after_Jan2010_count'] =count_2['after_Jan2010_count']

In [179]:
business_now = count_1

In [180]:
join = business_panda_updated

In [181]:
users_sum_grouped=business_now.groupby(['business_id'])['users_sum'].min().reset_index(name='users_sum')
right=join
left=users_sum_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

ratings_avg_grouped=business_now.groupby(['business_id'])['ratings_avg'].min().reset_index(name='ratings_avg')
right=join
left=ratings_avg_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

In [182]:
jan2015_grouped=business_now.groupby(['business_id'])['after_Jan2015'].min().reset_index(name='after_Jan2015_count')
right=join
left=after_Jan2015_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

jun2014_grouped=business_now.groupby(['business_id'])['after_Jun2014'].min().reset_index(name='after_Jun2014_count')
right=join
left=after_Jun2014_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

jan2014_grouped=business_now.groupby(['business_id'])['after_Jan2014'].min().reset_index(name='after_Jan2014_count')
right=join
left=after_Jan2014_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

jan2010_grouped=business_now.groupby(['business_id'])['after_Jan2010'].min().reset_index(name='after_Jan2010_count')
right=join
left=after_Jan2010_grouped
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

In [183]:
join

,business_id,after_Jan2010_count,after_Jan2014_count,after_Jun2014_count,after_Jan2015_count,ratings_avg_x,users_sum_x,review_count,stars,categories,...,attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open,users_sum_y,ratings_avg_y
0,01a4YOjiBbPUBxvgX6F7VQ,4,2,2,0,5.000000,3.810000,3,5.0,"[Acupuncture, Traditional Chinese Medicine, He...",...,NaN,NaN,NaN,NaN,NaN,NaN,[],True,753.435000,4.065816
1,04ogq23JnzauhezrWEXG_g,41,12,9,3,3.927011,68.275000,48,4.0,"[Caterers, Event Planning & Services, Barbeque...",...,False,False,False,False,False,False,[Banksville],True,753.435000,4.065816
2,06SQoq5EV4XAY-yKrEtCQg,4,1,0,0,3.937124,8.588333,5,3.0,"[Hotels & Travel, Car Rental]",...,NaN,NaN,NaN,NaN,NaN,NaN,[Downtown],True,133.095000,3.177304
3,079g3KpS_7j_Zw53JmD9Vg,67,20,14,5,3.817627,136.533333,74,3.5,"[American (Traditional), Restaurants]",...,False,False,False,True,False,False,[Shadyside],True,79.181667,4.126460
4,08Z2A6u4kmaAZYeecwA4qw,18,7,6,5,4.970899,34.363333,19,5.0,"[Arts & Entertainment, Art Galleries, Shopping]",...,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True,56.810000,3.672476
5,08arvIeYVIi4leIuFGhXBw,16,11,10,5,4.628873,21.300000,16,4.5,"[Tires, Automotive]",...,NaN,NaN,NaN,NaN,NaN,NaN,[Bellevue],True,11.088333,4.573275
6,08lbwQLMLaZNHRGIo5WJtQ,7,7,6,1,2.961170,23.306667,7,3.0,"[Shopping, Drugstores]",...,NaN,NaN,NaN,NaN,NaN,NaN,[Oakland],True,753.435000,4.065816
7,09r7xwM1OTEM2hnAM8WPFA,42,17,16,7,3.028215,58.716667,50,3.0,"[Italian, Comfort Food, Asian Fusion, Restaura...",...,False,False,False,False,False,False,[Oakland],True,208.990000,3.938123
8,0DlGOTI4xXLgF2eEm_E82Q,29,10,6,2,3.672476,56.810000,33,3.5,"[Hotels & Travel, Event Planning & Services, H...",...,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True,68.275000,3.927011
9,0E72Z_84LMrqLFAEJ276pA,3,0,0,0,4.700803,2.490000,3,4.5,"[Tobacco Shops, Shopping]",...,NaN,NaN,NaN,NaN,NaN,NaN,[],True,271.410000,4.078829


In [185]:
join.to_csv("PAbusiness_training_cluster.csv", encoding='utf-8')

In [286]:
join.columns.values

array(['business_id', 'clusters', 'after_Jan2010_count',
       'after_Jan2014_count', 'after_Jun2014_count', 'after_Jan2015_count',
       'ratings_avg', 'users_sum', 'categories', 'city', 'state', 'name',
       'review_count', 'stars', 'latitude', 'longitude',
       'attributes[Alcohol]', 'attributes[Ambience][casual]',
       'attributes[Ambience][classy]', 'attributes[Ambience][divey]',
       'attributes[Ambience][hipster]', 'attributes[Ambience][intimate]',
       'attributes[Ambience][romantic]', 'attributes[Ambience][touristy]',
       'attributes[Ambience][trendy]', 'attributes[Ambience][upscale]',
       'attributes[Delivery]', 'neighborhoods', 'open'], dtype=object)

In [186]:
PA_clustered = pd.read_csv('PA_business_clusters.csv')

In [188]:
PA_clustered['clusters'].max()

558

In [192]:
PA_clustered

,Unnamed: 0,business_id,lat,lon,clusters
0,1,01a4YOjiBbPUBxvgX6F7VQ,40.381770,-80.045186,1
1,2,04ogq23JnzauhezrWEXG_g,40.421516,-80.029730,2
2,3,06SQoq5EV4XAY-yKrEtCQg,40.443227,-80.004835,3
3,4,079g3KpS_7j_Zw53JmD9Vg,40.451096,-79.933261,4
4,5,08Z2A6u4kmaAZYeecwA4qw,40.458077,-80.009805,5
5,6,08arvIeYVIi4leIuFGhXBw,40.486936,-80.054628,6
6,7,08lbwQLMLaZNHRGIo5WJtQ,40.451886,-79.951931,7
7,8,09r7xwM1OTEM2hnAM8WPFA,40.441879,-79.956889,8
8,9,0DlGOTI4xXLgF2eEm_E82Q,40.448564,-80.004616,9
9,10,0E72Z_84LMrqLFAEJ276pA,40.481752,-79.850489,10


In [196]:
business_clusters = pd.DataFrame.from_csv('sunil_clusters.csv')

In [197]:
business_clusters

,business_id,latitude,longitude,clusters
1,0zO219KZJf-5QsRNRMmELQ,40.400387,-80.044015,1
2,13aQBVZEbO5MzGrmDJ0IVw,40.441542,-79.956821,2
3,3zgswf_NfBJpeAoWeTkgMA,40.458267,-79.942274,3
4,4S5CdZawGJfP3vtR4eAQ1g,40.440430,-79.994880,4
5,9lRJhLrLOgh0ZGhqsjBmLA,40.445689,-80.018228,5
6,FsY-8nYOCXyj9FoVxbjzTQ,40.471965,-80.037607,6
7,H27egCbna1FH8X8HFpLD_A,40.486969,-79.884861,7
8,IBQ-URinASf2z7_2Yt_LNg,40.451770,-80.016201,8
9,JEiSU3Kv6ZaS2y3AyT7_OA,40.407213,-80.086786,9
10,Mj7OA90Rx8Ts5G96X5pHCw,40.439187,-79.997901,4


In [198]:
right=join
left=business_clusters
join=left.merge(right, left_on='business_id', right_on='business_id', how='inner')

In [200]:
cols = join.columns.tolist()
cols

['business_id',
 'latitude_x',
 'longitude_x',
 'clusters',
 'after_Jan2010_count',
 'after_Jan2014_count',
 'after_Jun2014_count',
 'after_Jan2015_count',
 'ratings_avg_x',
 'users_sum_x',
 'review_count',
 'stars',
 'categories',
 'city',
 'state',
 'full_address',
 'latitude_y',
 'longitude_y',
 'name',
 'attributes[Alcohol]',
 'attributes[Ambience][casual]',
 'attributes[Ambience][classy]',
 'attributes[Ambience][divey]',
 'attributes[Ambience][hipster]',
 'attributes[Ambience][intimate]',
 'attributes[Ambience][romantic]',
 'attributes[Ambience][touristy]',
 'attributes[Ambience][trendy]',
 'attributes[Ambience][upscale]',
 'attributes[Delivery]',
 'neighborhoods',
 'open',
 'users_sum_y',
 'ratings_avg_y']

In [203]:
join.columns.values

array(['business_id', 'latitude_x', 'longitude_x', 'clusters',
       'after_Jan2010_count', 'after_Jan2014_count', 'after_Jun2014_count',
       'after_Jan2015_count', 'ratings_avg_x', 'users_sum_x',
       'review_count', 'stars', 'categories', 'city', 'state',
       'full_address', 'latitude_y', 'longitude_y', 'name',
       'attributes[Alcohol]', 'attributes[Ambience][casual]',
       'attributes[Ambience][classy]', 'attributes[Ambience][divey]',
       'attributes[Ambience][hipster]', 'attributes[Ambience][intimate]',
       'attributes[Ambience][romantic]', 'attributes[Ambience][touristy]',
       'attributes[Ambience][trendy]', 'attributes[Ambience][upscale]',
       'attributes[Delivery]', 'neighborhoods', 'open', 'users_sum_y',
       'ratings_avg_y'], dtype=object)

In [205]:
join.columns = ['business_id', 'latitude', 'longitude', 'clusters','after_Jan2010_count', 'after_Jan2014_count', 'after_Jun2014_count',\
                'after_Jan2015_count', 'ratings_avg', 'users_sum', 'review_count', 'stars', 'categories', 'city', 'state',\
                'full_address', 'latitude_y', 'longitude_y', 'name','attributes[Alcohol]', 'attributes[Ambience][casual]',\
                'attributes[Ambience][classy]', 'attributes[Ambience][divey]','attributes[Ambience][hipster]', 'attributes[Ambience][intimate]',\
                'attributes[Ambience][romantic]', 'attributes[Ambience][touristy]','attributes[Ambience][trendy]', 'attributes[Ambience][upscale]',\
                'attributes[Delivery]', 'neighborhoods', 'open', 'users_sum_y','ratings_avg_y']

In [211]:
join.head()

,business_id,latitude,longitude,clusters,after_Jan2010_count,after_Jan2014_count,after_Jun2014_count,after_Jan2015_count,ratings_avg,users_sum,...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,0zO219KZJf-5QsRNRMmELQ,40.400387,-80.044015,1,1,1,1,1,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Banksville],True
1,3zgswf_NfBJpeAoWeTkgMA,40.458267,-79.942274,3,39,13,8,4,3.673221,54.925000,...,False,False,False,False,False,False,False,False,[Bloomfield],True
2,4S5CdZawGJfP3vtR4eAQ1g,40.440430,-79.994880,4,53,18,16,7,3.678945,110.225000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Downtown],True
3,9lRJhLrLOgh0ZGhqsjBmLA,40.445689,-80.018228,5,3,2,2,1,5.000000,13.610000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True
4,FsY-8nYOCXyj9FoVxbjzTQ,40.471965,-80.037607,6,3,1,1,0,2.986005,6.788333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],False


In [212]:
join=join[['business_id',
 'clusters',
 'after_Jan2010_count',
 'after_Jan2014_count',
 'after_Jun2014_count',
 'after_Jan2015_count',
 'ratings_avg',
 'users_sum',
 'categories',
 'city',
 'state',
 'name',
 'review_count',
 'stars',
 'latitude',
 'longitude',
 'attributes[Alcohol]',
 'attributes[Ambience][casual]',
 'attributes[Ambience][classy]',
 'attributes[Ambience][divey]',
 'attributes[Ambience][hipster]',
 'attributes[Ambience][intimate]',
 'attributes[Ambience][romantic]',
 'attributes[Ambience][touristy]',
 'attributes[Ambience][trendy]',
 'attributes[Ambience][upscale]',
 'attributes[Delivery]',
 'neighborhoods',
 'open']]

In [213]:
join.head()

,business_id,clusters,after_Jan2010_count,after_Jan2014_count,after_Jun2014_count,after_Jan2015_count,ratings_avg,users_sum,categories,city,...,attributes[Ambience][divey],attributes[Ambience][hipster],attributes[Ambience][intimate],attributes[Ambience][romantic],attributes[Ambience][touristy],attributes[Ambience][trendy],attributes[Ambience][upscale],attributes[Delivery],neighborhoods,open
0,0zO219KZJf-5QsRNRMmELQ,1,1,1,1,1,1.000000,1.000000,[],Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Banksville],True
1,3zgswf_NfBJpeAoWeTkgMA,3,39,13,8,4,3.673221,54.925000,"[Pubs, Bars, American (Traditional), Nightlife...",Pittsburgh,...,False,False,False,False,False,False,False,False,[Bloomfield],True
2,4S5CdZawGJfP3vtR4eAQ1g,4,53,18,16,7,3.678945,110.225000,"[Hotels & Travel, Event Planning & Services, H...",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Downtown],True
3,9lRJhLrLOgh0ZGhqsjBmLA,5,3,2,2,1,5.000000,13.610000,"[Museums, Arts & Entertainment]",Chateau,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],True
4,FsY-8nYOCXyj9FoVxbjzTQ,6,3,1,1,0,2.986005,6.788333,"[Bars, Gay Bars, Nightlife]",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[North Side],False


In [214]:
business_regression = join

In [216]:
business_regression.to_csv('PAbusiness_regression_input.csv')

In [217]:
business_regression_model = pd.DataFrame.from_csv('PAregression_output_model.csv')

In [220]:
business_regression_model

,business_id,clusters,after_Jan2010_count,after_Jan2014_count,after_Jun2014_count,after_Jan2015_count,ratings_avg,users_sum,categories,city,...,attributes.Ambience..romantic.,attributes.Ambience..touristy.,attributes.Ambience..trendy.,attributes.Ambience..upscale.,attributes.Delivery.,neighborhoods,open,regression_value,weighted_regression_value,predicted_value
1,0zO219KZJf-5QsRNRMmELQ,1,1,1,1,1,1.000000,1.000000,[],Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,[Banksville],True,6.0,1.000000,5.765233
2,3zgswf_NfBJpeAoWeTkgMA,3,39,13,8,4,3.673221,54.925000,"[Pubs, Bars, American (Traditional), Nightlife...",Pittsburgh,...,False,False,False,False,False,[Bloomfield],True,208.0,201.751667,189.580900
3,4S5CdZawGJfP3vtR4eAQ1g,4,53,18,16,7,3.678945,110.225000,"[Hotels & Travel, Event Planning & Services, H...",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,[Downtown],True,213.5,405.511667,221.670267
4,9lRJhLrLOgh0ZGhqsjBmLA,5,3,2,2,1,5.000000,13.610000,"[Museums, Arts & Entertainment]",Chateau,...,NaN,NaN,NaN,NaN,NaN,[North Side],True,15.0,68.050000,18.194600
5,FsY-8nYOCXyj9FoVxbjzTQ,6,3,1,1,0,2.986005,6.788333,"[Bars, Gay Bars, Nightlife]",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,[North Side],False,18.0,20.270000,16.119300
6,H27egCbna1FH8X8HFpLD_A,7,9,5,5,1,2.673560,11.686667,"[Tires, Automotive, Auto Repair, Oil Change St...",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,[],True,18.0,31.245000,27.517950
7,IBQ-URinASf2z7_2Yt_LNg,8,3,2,2,2,5.000000,2.820000,"[Active Life, Pilates, Fitness & Instruction]",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,[North Side],True,15.0,14.100000,15.217067
8,JEiSU3Kv6ZaS2y3AyT7_OA,9,7,3,3,2,3.600169,11.863333,"[Fast Food, Pizza, Restaurants]",Carnegie,...,NaN,NaN,NaN,NaN,True,[Carnegie],True,36.0,42.710000,33.095517
9,Mj7OA90Rx8Ts5G96X5pHCw,4,2,1,1,1,5.000000,1.993333,"[Hair Salons, Day Spas, Nail Salons, Beauty & ...",Pittsburgh,...,NaN,NaN,NaN,NaN,NaN,[Downtown],True,15.0,9.966667,14.903467
10,P9zs1E1Z4JqIR-zPmD1URw,10,61,17,11,3,3.811347,91.305000,"[Pubs, Bars, Nightlife, Dance Clubs]",Lawrenceville,...,False,False,False,False,NaN,[Lawrenceville],True,264.0,347.995000,269.566750


In [223]:
def category_ratings(busniess_clusters):
    negcluster = busniess_clusters[busniess_clusters['ratings_avg']<3.5].reset_index()
    poscluster = busniess_clusters[busniess_clusters['ratings_avg']>=3.5].reset_index()

    pncaty={}

    for i in range(len(poscluster['categories'])):
        foodlist = poscluster['categories'][i].strip("[]").split(',')
        wt = poscluster['predicted_value'][i]
        for word in foodlist:
            word = word.rstrip(',')
            pncaty[word]=pncaty.get(word,0)+wt
            #print word


    for i in range(len(negcluster['categories'])):
        foodlist = negcluster['categories'][i].strip("[]").split()
        wt = -(negcluster['predicted_value'][i])
        for word in foodlist:
            word = word.rstrip(',')
            pncaty[word]=pncaty.get(word,0)+wt
    return pncaty

In [224]:
business_regression_model_sort = business_regression_model.groupby(['clusters']).apply(category_ratings)

In [225]:
def category_frequencies(busniess_clusters):
    poscluster = busniess_clusters.reset_index()
    pncaty={}

    for i in range(len(poscluster['categories'])):
        foodlist = poscluster['categories'][i].strip("[]").split(',')
        for word in foodlist:
            word = word.rstrip(',')
            pncaty[word]=pncaty.get(word,0)+1
    #print word
    #sorted_words = (sorted(pncaty.items(), key=operator.itemgetter(1), reverse = True))        
    return pncaty

In [227]:
clusters_rating = business_regression_model.groupby(['clusters']).apply(category_ratings)


In [228]:
clusters_frequency = business_regression_model.groupby(['clusters']).apply(category_frequencies)


In [229]:
clusters_frequency

clusters
1      {u'': 1, u' Ice Cream & Frozen Yogurt': 1, u'B...
2      {u' Sewing & Alterations': 1, u' Dentists': 1,...
3      {u' Auto Repair': 1, u' Dentists': 9, u'Tapas/...
4      {u' Auto Repair': 1, u' Candy Stores': 2, u' D...
5      {u' Stadiums & Arenas': 2, u' Shopping': 1, u'...
6        {u' Gay Bars': 1, u'Bars': 1, u' Nightlife': 1}
7      {u' Auto Repair': 1, u' Ice Cream & Frozen Yog...
8      {u' Payroll Services': 1, u' Dentists': 1, u'A...
9      {u'': 1, u' Sewing & Alterations': 1, u'Americ...
10     {u' Musical Instrument Services': 1, u'Arts & ...
11     {u'Bars': 1, u' Nightlife': 1, u'American (New...
12     {u' Cards & Stationery': 1, u' Ice Cream & Fro...
13     {u' Ice Cream & Frozen Yogurt': 1, u' Shopping...
14     {u' Shopping': 1, u' Carpet Cleaning': 1, u' H...
15     {u' Candy Stores': 2, u' Dentists': 1, u' Musi...
16     {u'Bars': 1, u'Shopping': 1, u' Shopping': 1, ...
17     {u' Car Dealers': 1, u'Health & Medical': 1, u...
18                    

In [230]:
clusters_rating

clusters
1      {u'Salons': -68.27725, u' Ice Cream & Frozen Y...
2      {u'Notaries': -11.7339, u'Stands': -21.5505333...
3      {u'Salons': -225.4657, u' Dentists': 105.59183...
4      {u'Vitamins': -20.0717, u'Drugstores': -32.591...
5      {u' Stadiums & Arenas': 645.5889, u' Shopping'...
6      {u'Bars': -32.2386, u'Nightlife': -16.1193, u'...
7      {u'Salons': -4.77503333333, u' Shopping': 83.9...
8      {u' Payroll Services': 25.21235, u' Dentists':...
9      {u'Spirits': -9.34486666667, u'American (New)'...
10     {u'Service': -15.4328333333, u' Musical Instru...
11     {u'Bars': 41.16325, u'Grocery': -14.6520666667...
12     {u'Salons': -77.7509666667, u'Arts': -9.347533...
13     {u'Arts': -50.9757166667, u' Ice Cream & Froze...
14     {u'Shopping': -10.7828, u'Garden': -10.7828, u...
15     {u' Candy Stores': 65.3748, u'Service': -25.48...
16     {u'Bars': 39.3142333333, u'Shopping': 59.52955...
17     {u' Car Dealers': 28.1889833333, u'Health & Me...
18     {u' Parks': 43.

In [231]:
clusters_rating_cats =  pd.Series.to_frame(clusters_rating)
clusters_rating_cats = clusters_rating_cats[0].reset_index()
clusters_rating_cats.columns = ['cluster', 'cluster_ratings']

In [232]:
clusters_new_ratings=clusters_rating_cats

In [233]:
clusters_rating_cats

,cluster,cluster_ratings
0,1,"{u'Salons': -68.27725, u' Ice Cream & Frozen Y..."
1,2,"{u'Notaries': -11.7339, u'Stands': -21.5505333..."
2,3,"{u'Salons': -225.4657, u' Dentists': 105.59183..."
3,4,"{u'Vitamins': -20.0717, u'Drugstores': -32.591..."
4,5,"{u' Stadiums & Arenas': 645.5889, u' Shopping'..."
5,6,"{u'Bars': -32.2386, u'Nightlife': -16.1193, u'..."
6,7,"{u'Salons': -4.77503333333, u' Shopping': 83.9..."
7,8,"{u' Payroll Services': 25.21235, u' Dentists':..."
8,9,"{u'Spirits': -9.34486666667, u'American (New)'..."
9,10,"{u'Service': -15.4328333333, u' Musical Instru..."


In [234]:
clusters_frequency_cats =  pd.Series.to_frame(clusters_frequency)
clusters_frequency_cats = clusters_frequency_cats[0].reset_index()
clusters_frequency_cats.columns = ['cluster', 'cluster_frequency']
clusters_new_freq=clusters_frequency_cats

In [235]:
clusters_new_freq

,cluster,cluster_frequency
0,1,"{u'': 1, u' Ice Cream & Frozen Yogurt': 1, u'B..."
1,2,"{u' Sewing & Alterations': 1, u' Dentists': 1,..."
2,3,"{u' Auto Repair': 1, u' Dentists': 9, u'Tapas/..."
3,4,"{u' Auto Repair': 1, u' Candy Stores': 2, u' D..."
4,5,"{u' Stadiums & Arenas': 2, u' Shopping': 1, u'..."
5,6,"{u' Gay Bars': 1, u'Bars': 1, u' Nightlife': 1}"
6,7,"{u' Auto Repair': 1, u' Ice Cream & Frozen Yog..."
7,8,"{u' Payroll Services': 1, u' Dentists': 1, u'A..."
8,9,"{u'': 1, u' Sewing & Alterations': 1, u'Americ..."
9,10,"{u' Musical Instrument Services': 1, u'Arts & ..."


In [238]:
freq_list=[]
freq__top_sorted=[]
freq__low_sorted=[]

for i in range(len(clusters_new_freq['cluster'])):
    words = (sorted(clusters_new_freq['cluster_frequency'][i].items(), key=operator.itemgetter(1), reverse = True))
    freq__top_sorted.append(words)
    low_words=(sorted(clusters_new_freq['cluster_frequency'][i].items(), key=operator.itemgetter(1)))
    freq__low_sorted.append(low_words)
    wordlist=[]
    for word in words:
        wordlist.append(word[0])
        if(len(wordlist)>9):
            break
    freq_list.append(wordlist)

In [239]:
freq_list

[[' Restaurants',
  'Food',
  'Pizza',
  ' Beauty & Spas',
  ' Sandwiches',
  '',
  ' Ice Cream & Frozen Yogurt',
  'Beauty & Spas',
  'Fast Food',
  'Indian'],
 [' Restaurants',
  'Food',
  ' Shopping',
  ' Nightlife',
  ' Beauty & Spas',
  ' Bars',
  ' Coffee & Tea',
  ' Sandwiches',
  'Hair Salons',
  ' Chinese'],
 [' Restaurants',
  'Food',
  ' Nightlife',
  ' Beauty & Spas',
  ' Shopping',
  'Bars',
  'Pizza',
  ' Health & Medical',
  ' Bars',
  ' Coffee & Tea'],
 [' Restaurants',
  'Food',
  ' Nightlife',
  ' Shopping',
  'Bars',
  ' Coffee & Tea',
  'Hotels & Travel',
  'American (New)',
  ' Italian',
  ' Sandwiches'],
 ['Arts & Entertainment',
  ' Arts & Entertainment',
  ' Restaurants',
  ' Nightlife',
  'Museums',
  ' Stadiums & Arenas',
  ' Venues & Event Spaces',
  ' American (New)',
  ' Event Planning & Services',
  ' Shopping'],
 [' Gay Bars', 'Bars', ' Nightlife'],
 [' Restaurants',
  'Food',
  ' Shopping',
  'Burgers',
  ' Grocery',
  ' American (New)',
  ' Fast Food',


In [240]:
clusters_frequency_cats['top_freq']="[]"
for i in range(len(clusters_frequency_cats['cluster'])):
       clusters_frequency_cats['top_freq'][i] = freq_list[i]

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [241]:
clusters_frequency_cats

,cluster,cluster_frequency,top_freq
0,1,"{u'': 1, u' Ice Cream & Frozen Yogurt': 1, u'B...","[ Restaurants, Food, Pizza, Beauty & Spas, S..."
1,2,"{u' Sewing & Alterations': 1, u' Dentists': 1,...","[ Restaurants, Food, Shopping, Nightlife, B..."
2,3,"{u' Auto Repair': 1, u' Dentists': 9, u'Tapas/...","[ Restaurants, Food, Nightlife, Beauty & Spa..."
3,4,"{u' Auto Repair': 1, u' Candy Stores': 2, u' D...","[ Restaurants, Food, Nightlife, Shopping, Ba..."
4,5,"{u' Stadiums & Arenas': 2, u' Shopping': 1, u'...","[Arts & Entertainment, Arts & Entertainment, ..."
5,6,"{u' Gay Bars': 1, u'Bars': 1, u' Nightlife': 1}","[ Gay Bars, Bars, Nightlife]"
6,7,"{u' Auto Repair': 1, u' Ice Cream & Frozen Yog...","[ Restaurants, Food, Shopping, Burgers, Groc..."
7,8,"{u' Payroll Services': 1, u' Dentists': 1, u'A...","[ Restaurants, Burgers, Sandwiches, American ..."
8,9,"{u'': 1, u' Sewing & Alterations': 1, u'Americ...","[ Restaurants, Food, Diners, Nightlife, Iris..."
9,10,"{u' Musical Instrument Services': 1, u'Arts & ...","[ Restaurants, Shopping, Food, Nightlife, F..."


In [243]:
rating_list=[]
rating__top_sorted=[]
rating__low_sorted=[]

for i in range(len(clusters_new_ratings['cluster'])):
    words = (sorted(clusters_new_ratings['cluster_ratings'][i].items(), key=operator.itemgetter(1), reverse = True))
    rating__top_sorted.append(words)
    low_words=(sorted(clusters_new_ratings['cluster_ratings'][i].items(), key=operator.itemgetter(1)))
    rating__low_sorted.append(low_words)
    wordlist=[]
    for word in words:
        wordlist.append(word[0])
        if(len(wordlist)>9):
            break
    rating_list.append(wordlist)

In [244]:
rating_list

[[' Restaurants',
  ' Grocery',
  ' Auto Detailing',
  'Automotive',
  ' Sandwiches',
  'Food',
  'Italian',
  ' Pizza',
  'American (Traditional)',
  ' Ice Cream & Frozen Yogurt'],
 [' Restaurants',
  'Food',
  'American (New)',
  ' Ice Cream & Frozen Yogurt',
  ' Asian Fusion',
  ' Arts & Entertainment',
  'Mexican',
  ' Tea Rooms',
  'American (Traditional)',
  'Delis'],
 [' Restaurants',
  ' Nightlife',
  ' American (Traditional)',
  'Bars',
  'Thai',
  ' Beauty & Spas',
  'Burgers',
  ' Barbeque',
  ' Mediterranean',
  ' American (New)'],
 [' Restaurants',
  'American (New)',
  ' American (New)',
  ' Nightlife',
  ' Italian',
  ' Pizza',
  'Steakhouses',
  ' Sandwiches',
  ' Event Planning & Services',
  'Food'],
 ['Arts & Entertainment',
  ' Venues & Event Spaces',
  ' Event Planning & Services',
  ' Stadiums & Arenas',
  ' Arts & Entertainment',
  'Museums',
  ' Nightlife',
  ' Music Venues',
  'Professional Sports Teams',
  ' Italian'],
 ['Nightlife', 'Gay', 'Bars'],
 [' Restau

In [245]:
clusters_rating_cats['top_categories']="[]"
for i in range(len(clusters_rating_cats['cluster'])):
       clusters_rating_cats['top_categories'][i] = rating_list[i]

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [246]:
clusters_rating_cats

,cluster,cluster_ratings,top_categories
0,1,"{u'Salons': -68.27725, u' Ice Cream & Frozen Y...","[ Restaurants, Grocery, Auto Detailing, Auto..."
1,2,"{u'Notaries': -11.7339, u'Stands': -21.5505333...","[ Restaurants, Food, American (New), Ice Crea..."
2,3,"{u'Salons': -225.4657, u' Dentists': 105.59183...","[ Restaurants, Nightlife, American (Traditio..."
3,4,"{u'Vitamins': -20.0717, u'Drugstores': -32.591...","[ Restaurants, American (New), American (New)..."
4,5,"{u' Stadiums & Arenas': 645.5889, u' Shopping'...","[Arts & Entertainment, Venues & Event Spaces,..."
5,6,"{u'Bars': -32.2386, u'Nightlife': -16.1193, u'...","[Nightlife, Gay, Bars]"
6,7,"{u'Salons': -4.77503333333, u' Shopping': 83.9...","[ Restaurants, American (New), Burgers, Vege..."
7,8,"{u' Payroll Services': 25.21235, u' Dentists':...","[ Restaurants, Thai, Burgers, Gastropubs, Sou..."
8,9,"{u'Spirits': -9.34486666667, u'American (New)'...","[ Restaurants, Indian, Food, Coffee & Tea, Am..."
9,10,"{u'Service': -15.4328333333, u' Musical Instru...","[ Restaurants, American (New), Food, Food, B..."


In [248]:
top_rated_businesses_clustered=clusters_rating_cats[['top_categories']]

In [249]:
top_rated_businesses_clustered.to_csv("PAbusiness_top_rated_clustered.csv", encoding='utf-8')

In [250]:
business_arules = pd.read_csv('PAbusiness_top_rated_arules.csv')

In [251]:
business_arules.head()

,Unnamed: 0,lhs,Unnamed: 2,rhs,support,confidence,lift
0,74,Pubs,=>,Nightlife,0.023423,0.866667,8.745455
1,249,"American (New),Bars",=>,Nightlife,0.021622,0.923077,9.314685
2,63,"Hotels""",=>,&,0.014414,1.000000,12.613636
3,72,Wine & Spirits,=>,Beer,0.014414,0.800000,37.000000
4,73,Fitness & Instruction,=>,Gyms,0.014414,0.800000,34.153846


In [253]:
del business_arules['Unnamed: 2']

In [254]:
business_arules.head()

,lhs,rhs,support,confidence,lift
0,Pubs,Nightlife,0.023423,0.866667,8.745455
1,"American (New),Bars",Nightlife,0.021622,0.923077,9.314685
2,"Hotels""",&,0.014414,1.000000,12.613636
3,Wine & Spirits,Beer,0.014414,0.800000,37.000000
4,Fitness & Instruction,Gyms,0.014414,0.800000,34.153846


In [255]:
business_arules = business_arules.reset_index()

In [295]:
business_arules[business_arules['index']>372]

,index,rule_value,rule_key,support,confidence,lift
373,373,"American (New),Bars,Food",Nightlife,0.005405,1.00,10.090909
374,374,"American (New),Food,Nightlife",Bars,0.005405,0.75,9.250000
375,375,"Auto,Oil,Stations,Tires",Automotive,0.005405,1.00,15.416667
376,376,"Auto,Automotive,Oil,Stations",Tires,0.005405,1.00,61.666667
377,377,"Auto,Automotive,Oil,Tires",Stations,0.005405,1.00,79.285714
378,378,"Automotive,Oil,Stations,Tires",Auto,0.005405,1.00,69.375000
379,379,"Auto,Automotive,Stations,Tires",Oil,0.005405,1.00,185.000000
380,380,"American (New),Bars,Gastropubs,Pubs",Nightlife,0.005405,1.00,10.090909
381,381,"American (New),Gastropubs,Nightlife,Pubs",Bars,0.005405,1.00,12.333333
382,382,"Bars,Gastropubs,Nightlife,Pubs",American (New),0.005405,1.00,17.903226


In [290]:
business_arules.columns = ['index', 'rule_value','rule_key','support','confidence','lift']

In [291]:
business_arules.head()

,index,rule_value,rule_key,support,confidence,lift
0,0,Pubs,Nightlife,0.023423,0.866667,8.745455
1,1,"American (New),Bars",Nightlife,0.021622,0.923077,9.314685
2,2,"Hotels""",&,0.014414,1.000000,12.613636
3,3,Wine & Spirits,Beer,0.014414,0.800000,37.000000
4,4,Fitness & Instruction,Gyms,0.014414,0.800000,34.153846


In [259]:
rule_dict={}

for i in range(len(business_arules['cluster'])):
    key = business_arules['rule_key'][i].strip()
    
    if key not in rule_dict:
        values =  []
        rule_dict[key] = values
    else:
        values = rule_dict[key]
        
    for word in business_arules['rule_value'][i].split(','):
        word = word.strip()
        if word not in values:
            values.append(word)  

In [265]:
opportunity = {}
for key in rule_dict.keys():
    cid={}
    for i in range(len(freq_list)):
        if key in freq_list[i]:
            rule_values = rule_dict[key]
            freq_values = freq_list[i]
            caty = []
            for rule_value in rule_values:
                if rule_value not in freq_values:
                    caty.append(rule_value)            
            cid[i]=caty

    opportunity[key] = cid

In [272]:
opportunity

{'Active Life': {17: ['Gyms',
   'Restaurants',
   'Yoga',
   'Fitness & Instruction',
   'Coffee & Tea'],
  25: ['Gyms', 'Restaurants', 'Yoga', 'Fitness & Instruction', 'Coffee & Tea'],
  26: ['Gyms', 'Restaurants', 'Yoga', 'Fitness & Instruction', 'Coffee & Tea'],
  33: ['Gyms', 'Restaurants', 'Yoga', 'Fitness & Instruction', 'Coffee & Tea'],
  56: ['Gyms', 'Restaurants', 'Yoga', 'Fitness & Instruction', 'Coffee & Tea'],
  59: ['Gyms', 'Restaurants', 'Yoga', 'Fitness & Instruction', 'Coffee & Tea'],
  66: ['Gyms', 'Restaurants', 'Yoga', 'Fitness & Instruction', 'Coffee & Tea'],
  75: ['Gyms', 'Restaurants', 'Yoga', 'Fitness & Instruction', 'Coffee & Tea'],
  78: ['Gyms', 'Restaurants', 'Yoga', 'Fitness & Instruction', 'Coffee & Tea'],
  110: ['Gyms',
   'Restaurants',
   'Yoga',
   'Fitness & Instruction',
   'Coffee & Tea'],
  121: ['Gyms',
   'Restaurants',
   'Yoga',
   'Fitness & Instruction',
   'Coffee & Tea'],
  137: ['Gyms',
   'Restaurants',
   'Yoga',
   'Fitness & Instruct

In [274]:
business_pa_opportunities = pd.DataFrame.from_dict(opportunity.items())

In [305]:
business_pa_opportunities.columns = ['Category_key','Complementor_business']

In [303]:
business_pa_opportunities['Complementary_business'][8]

{468: ['Veterinarians']}

In [320]:
business_comp = pd.DataFrame()

In [321]:
business_comp['Complementor_business'] = business_pa_opportunities['Complementor_business']

In [322]:
business_comp

,Complementor_business
0,"{3: [u'Nightlife', u'Pubs', u'Gastropubs', u'S..."
1,"{0: [u'Barbers', u'Bars', u'Mediterranean', u'..."
2,{}
3,"{4: [], 9: [u'Museums'], 266: [u'Museums'], 11..."
4,"{97: [u'Diners', u'Food'], 34: [u'Diners', u'F..."
5,{}
6,{}
7,"{193: [u'American (New)', u'Italian'], 74: [u'..."
8,{468: [u'Veterinarians']}
9,{}


In [324]:
business_comp['Complementor_business'][59]

{12: ['Pubs',
  'American (New)',
  'Sports Bars',
  'Breakfast & Brunch',
  'Pizza',
  'Gastropubs',
  'Bars"',
  'Restaurants',
  '(Traditional)',
  'Coffee & Tea',
  'Steakhouses',
  'American',
  'Beauty & Spas',
  'Mediterranean',
  'Barbeque',
  'Cafes',
  'American (Traditional)',
  'Restaurants"',
  'Diners',
  'Sandwiches',
  'Shopping'],
 285: ['Pubs',
  'American (New)',
  'Bars',
  'Sports Bars',
  'Food',
  'Breakfast & Brunch',
  'Pizza',
  'Gastropubs',
  'Bars"',
  'Restaurants',
  '(Traditional)',
  'Coffee & Tea',
  'Steakhouses',
  'American',
  'Beauty & Spas',
  'Mediterranean',
  'Barbeque',
  'Cafes',
  'American (Traditional)',
  'Restaurants"',
  'Diners',
  'Sandwiches',
  'Shopping'],
 346: ['Pubs',
  'American (New)',
  'Bars',
  'Sports Bars',
  'Food',
  'Breakfast & Brunch',
  'Pizza',
  'Gastropubs',
  'Bars"',
  'Restaurants',
  '(Traditional)',
  'Coffee & Tea',
  'Steakhouses',
  'American',
  'Beauty & Spas',
  'Mediterranean',
  'Barbeque',
  'Cafes